### Neighborhood Battle-African Restaurant and Pub in London Part 2

In part 1 of this presentation i was able to extract the data use from wikipedia. The data was cleanup and foursquare used to identified and categorised venues and locations. In part 2 of this project cluster analysis is used to analysed data from part 1 of this poject  


In [ ]:
pip install sklearn

In [ ]:
pip install KMeans

  Created wheel for KMeans: filename=kmeans-1.0.2-cp37-cp37m-linux_x86_64.whl size=11759 sha256=697c602ae9c2ed798a52b9906d51a4e18fe9f9123126360380bada3e4a590c9f
  Stored in directory: /root/.cache/pip/wheels/10/05/77/2735fd2f7a0b4d86cb7a83be2a56807cf706d5d27f742c6482
Successfully built KMeans


In [ ]:
from sklearn.cluster import KMeans


In [ ]:
# Clustering African Borough

kclusters = 5

lon_grouped_clustering = lon_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(lon_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
# to change use .astype()

array([2, 0, 4, 4, 0, 4, 0, 0, 0, 0], dtype=int32)

In [ ]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

lon_merged = lon_com2

# merge lon_grouped with lon_com2 to add latitude/longitude for each neighborhood
lon_merged = lon_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Borough')

lon_merged.head() # check the last columns!

,Borough,African,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Southwark,47413,51.502922,-0.103458,3,Scenic Lookout,Hotel,Coffee Shop,Pub,Garden,Theater,Pizza Place,Wine Bar,Cocktail Bar,Plaza
4,Newham,37811,51.530000,0.029318,4,Pub,Hotel,Indian Restaurant,Café,Coffee Shop,Supermarket,Grocery Store,Platform,Park,Furniture / Home Store
0,Lambeth,35187,51.501301,-0.117287,3,Hotel,Bookstore,Theater,Plaza,Garden,Lounge,Art Museum,Scenic Lookout,Park,Street Food Gathering
8,Greenwich,35164,51.482084,-0.004542,0,Pub,Park,Bar,Garden,Coffee Shop,Brewery,Gym / Fitness Center,Café,Turkish Restaurant,Farmers Market
2,Lewisham,32025,51.462432,-0.010133,0,Pub,Coffee Shop,Café,Park,Gastropub,Garden,Bar,Gym / Fitness Center,Fish & Chips Shop,Restaurant


In [ ]:
lon_merged.drop(columns=['African'], inplace = True)
lon_merged.head()

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Southwark,51.502922,-0.103458,3,Scenic Lookout,Hotel,Coffee Shop,Pub,Garden,Theater,Pizza Place,Wine Bar,Cocktail Bar,Plaza
4,Newham,51.530000,0.029318,4,Pub,Hotel,Indian Restaurant,Café,Coffee Shop,Supermarket,Grocery Store,Platform,Park,Furniture / Home Store
0,Lambeth,51.501301,-0.117287,3,Hotel,Bookstore,Theater,Plaza,Garden,Lounge,Art Museum,Scenic Lookout,Park,Street Food Gathering
8,Greenwich,51.482084,-0.004542,0,Pub,Park,Bar,Garden,Coffee Shop,Brewery,Gym / Fitness Center,Café,Turkish Restaurant,Farmers Market
2,Lewisham,51.462432,-0.010133,0,Pub,Coffee Shop,Café,Park,Gastropub,Garden,Bar,Gym / Fitness Center,Fish & Chips Shop,Restaurant


In [ ]:
lon_merged=lon_merged.dropna()

In [ ]:
from matplotlib import cm
import matplotlib.colors as colors


In [ ]:
# ploting cluster in a map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lon_merged['Latitude'], lon_merged['Longitude'], lon_merged['Borough'], lon_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [ ]:
lon_merged['Cluster Labels'].value_counts()

0    10
4     6
3     2
2     1
1     1
Name: Cluster Labels, dtype: int64

Analyzing Clusters

In [ ]:
# cluster 1

lon_merged.loc[lon_merged['Cluster Labels'] == 0, lon_merged.columns[[1] + list(range(5, lon_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,51.482084,Park,Bar,Garden,Coffee Shop,Brewery,Gym / Fitness Center,Café,Turkish Restaurant,Farmers Market
2,51.462432,Coffee Shop,Café,Park,Gastropub,Garden,Bar,Gym / Fitness Center,Fish & Chips Shop,Restaurant
6,51.543240,Coffee Shop,Café,Bakery,Park,Restaurant,Brewery,Deli / Bodega,Farmers Market,Beer Bar
9,51.587930,Turkish Restaurant,Coffee Shop,Park,Café,Bakery,Italian Restaurant,Grocery Store,Greek Restaurant,Bar
14,51.653090,Coffee Shop,Grocery Store,Park,Pharmacy,Bookstore,Train Station,Golf Course,Pizza Place,Fish & Chips Shop
12,51.512655,Coffee Shop,Park,Italian Restaurant,Hotel,Café,Pizza Place,Burger Joint,Bakery,Persian Restaurant
13,51.457027,Park,Café,Coffee Shop,Pizza Place,Supermarket,Bakery,Grocery Store,Gastropub,French Restaurant
15,51.538429,Café,Coffee Shop,Bakery,Park,Canal,Pizza Place,Trail,Breakfast Spot,Bar
17,51.492038,Café,Bakery,Coffee Shop,Italian Restaurant,Park,Gastropub,Middle Eastern Restaurant,Gift Shop,Toy / Game Store
18,51.410870,Park,Sushi Restaurant,Bar,Café,Coffee Shop,Burger Joint,Italian Restaurant,Lebanese Restaurant,Gym / Fitness Center


In [ ]:
# cluster 2
lon_merged.loc[lon_merged['Cluster Labels'] == 1, lon_merged.columns[[1] + list(range(5, lon_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,51.198159,Gastropub,Vacation Rental,Farm,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Flea Market,Fish Market


In [ ]:
# cluster 3
lon_merged.loc[lon_merged['Cluster Labels'] == 2, lon_merged.columns[[1] + list(range(5, lon_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,51.554117,Supermarket,Pub,Gas Station,Park,Cosmetics Shop,Café,Food & Drink Shop,Platform,Furniture / Home Store


In [ ]:
# cluster 4
lon_merged.loc[lon_merged['Cluster Labels'] == 3, lon_merged.columns[[1] + list(range(5, lon_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,51.502922,Hotel,Coffee Shop,Pub,Garden,Theater,Pizza Place,Wine Bar,Cocktail Bar,Plaza
0,51.501301,Bookstore,Theater,Plaza,Garden,Lounge,Art Museum,Scenic Lookout,Park,Street Food Gathering


In [ ]:
# cluster 5
lon_merged.loc[lon_merged['Cluster Labels'] == 4, lon_merged.columns[[1] + list(range(5, lon_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,51.530000,Hotel,Indian Restaurant,Café,Coffee Shop,Supermarket,Grocery Store,Platform,Park,Furniture / Home Store
3,51.371305,Hotel,Park,Grocery Store,Coffee Shop,Café,Indian Restaurant,Fast Food Restaurant,South Indian Restaurant,Platform
7,51.652085,Supermarket,Coffee Shop,Grocery Store,Train Station,Turkish Restaurant,Gym / Fitness Center,Bar,Fish & Chips Shop,Indian Restaurant
5,52.018488,Clothing Store,Pub,Hotel,Fast Food Restaurant,Supermarket,Coffee Shop,Furniture / Home Store,Gym,Pizza Place
16,51.576320,Grocery Store,Park,Indian Restaurant,Pizza Place,Hotel,Coffee Shop,Restaurant,Turkish Restaurant,Bakery
19,51.542519,Pub,Grocery Store,Gym,Pharmacy,Clothing Store,Restaurant,Pizza Place,Supermarket,Italian Restaurant


The following are the highlights of the 5 clusters above:

Southwark is the borough with the highest African population, and it has more coffee shops, hotels, theaters, and pub, with no African brand name attached to them. Italian, Turkish, and Indian are common resturants at these Boroughs with high African population.

Pubs and coffeeshops are very popular in the south east London.

In cluster 2 and 5 is where top African restaurants, pubs, coffee shops and café can be fit in.
